In [1]:
import sys
import os
import wandb
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras
from wandb.keras import WandbCallback

from data import*

######## for correcting the error from argon
import tensorflow as tf
# Config to turn on JIT compilation
#config = tf.ConfigProto()
#config.graph_options.optimizer_options.global_jit_level = tf.OptimizerOptions.ON_1
#sess = tf.Session(config=config)
####################

from tensorflow.keras.models import load_model
model = load_model("./output/ThreeLevel_CT_Inhouse.hdf5")
sys.path.append(os.getcwd()+'/code')

2022-03-31 15:44:49.232850: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-03-31 15:44:49.287861: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:d8:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-03-31 15:44:49.288283: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-03-31 15:44:49.291798: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-03-31 15:44:49.296006: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-03-31 15:44:49.296593: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so

In [2]:
import sys
#print(sys.path)
print(len(model.layers)), #model.summary()

39


(None,)

In [3]:
#parameters for each of the protocols

RadialParams = { 'num_ep' : 60,
                'num_step':100,
                'num_layers': 18,
                'learning_rate' : 1e-4,
                'output_loc' : './NetworkOutput/Generic_Network_Testing/Radial',
                'model_loc_loss' : './output/ThreeLevel_CT_Inhouse_Radial_loss.hdf5',
                'model_loc' : './output/ThreeLevel_CT_Inhouse_Radial.hdf5',
                'train_loc' : './data/Radial/Train/',
                'val_loc' : './data/Radial/Val/',
                'test_loc' : './data/Radial/Test/Image/',
                'project' : 'ThreeLevel_CT_Inhouse_Radial'
               }

StormParams = {'num_ep' : 60,
                'num_step':100,
                'num_layers': 18,
                'learning_rate' : 1e-3,
                'output_loc' : './NetworkOutput/Generic_Network_Testing/Storm/',
                'model_loc' : './output/ThreeLevel_CT_Inhouse_Storm_Thresholded.hdf5',
                'train_loc' : './data/Storm/Train/',
                'val_loc' : './data/Storm/Val/',
                'test_loc' : './data/Storm/Test/TImage/',
                'project' : 'ThreeLevel_CT_Inhouse_Storm_train',
                'Image' : 'TImage',
                'Label' : 'Airway'
              }

USCParams = {'num_ep' : 35,
                'num_step':100,
                'num_layers': 20,
                'learning_rate' : 1e-4,
                'output_loc' : './NetworkOutput/Generic_Network_Testing/USC/',
                'model_loc' : './output/ThreeLevel_CT_Inhouse_USC.hdf5',
                'train_loc' : './data/USC/Segmentations/Train',
                'val_loc' : './data/USC/Segmentations/Val',
                'test_loc' : './data/USC/Segmentations/Test/Image',
                'project' : 'ThreeLevel_CT_Inhouse_USC',
                'Image' : 'Image',
                'Label' : 'Airway'
}

Params = StormParams
wandb.init(project=Params['project'], entity="subinek")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: subinek (use `wandb login --relogin` to force relogin)
/home/erattakulangara/.conda/envs/chestXSeg/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
#creating new folder path
if not os.path.exists(Params['output_loc']):
    os.makedirs(output_loc)
    print('done')


# Freezing layers
for i in range(0,Params['num_layers']):
    model.layers[i].trainable = False
    
start_val = Params['num_layers']
end_val = len(model.layers)
for i in range(start_val,end_val):
    model.layers[i].trainable = True

## non - sweep config

In [6]:

model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    fill_mode='nearest')

myGene = trainGenerator(2,Params['train_loc'],Params['Image'],Params['Label'],data_gen_args,save_to_dir = None)

myVal = trainGenerator(2,Params['val_loc'],Params['Image'],Params['Label'],data_gen_args,save_to_dir = None)

model_checkpoint = ModelCheckpoint(Params['model_loc'], monitor='val_loss',verbose=1, save_best_only=True)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=4, 
                                       verbose=1, mode='auto', min_lr=0.001)

h=model.fit_generator(myGene,validation_data = myVal,validation_steps = 1,steps_per_epoch=Params['num_step'],epochs=Params['num_ep'],callbacks=[model_checkpoint, reduce_lr, WandbCallback()])

Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Epoch 1/60


2022-03-31 15:46:47.287811: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2022-03-31 15:46:48.871445: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


100/100 [==============================] - ETA: 0s - loss: 0.0627 - accuracy: 0.9787Found 3 images belonging to 1 classes.
Found 3 images belonging to 1 classes.

Epoch 00001: val_loss improved from inf to 0.05926, saving model to ./output/ThreeLevel_CT_Inhouse_Storm_Thresholded.hdf5
100/100 [==============================] - 8s 77ms/step - loss: 0.0627 - accuracy: 0.9787 - val_loss: 0.0593 - val_accuracy: 0.9766 - lr: 1.0000e-04
Epoch 2/60
100/100 [==============================] - ETA: 0s - loss: 0.0296 - accuracy: 0.9875
Epoch 00002: val_loss did not improve from 0.05926
100/100 [==============================] - 5s 51ms/step - loss: 0.0296 - accuracy: 0.9875 - val_loss: 0.0593 - val_accuracy: 0.9802 - lr: 1.0000e-04
Epoch 3/60
 99/100 [============================>.] - ETA: 0s - loss: 0.0222 - accuracy: 0.9906
Epoch 00003: val_loss did not improve from 0.05926
100/100 [==============================] - 5s 50ms/step - loss: 0.0222 - accuracy: 0.9906 - val_loss: 0.0735 - val_accuracy

100/100 [==============================] - ETA: 0s - loss: 0.0079 - accuracy: 0.9967
Epoch 00028: val_loss did not improve from 0.05926
100/100 [==============================] - 5s 52ms/step - loss: 0.0079 - accuracy: 0.9967 - val_loss: 0.1354 - val_accuracy: 0.9752 - lr: 1.0000e-04
Epoch 29/60
100/100 [==============================] - ETA: 0s - loss: 0.0068 - accuracy: 0.9971
Epoch 00029: val_loss did not improve from 0.05926
100/100 [==============================] - 5s 51ms/step - loss: 0.0068 - accuracy: 0.9971 - val_loss: 0.1445 - val_accuracy: 0.9776 - lr: 1.0000e-04
Epoch 30/60
100/100 [==============================] - ETA: 0s - loss: 0.0068 - accuracy: 0.9971
Epoch 00030: val_loss did not improve from 0.05926
100/100 [==============================] - 5s 51ms/step - loss: 0.0068 - accuracy: 0.9971 - val_loss: 0.1485 - val_accuracy: 0.9757 - lr: 1.0000e-04
Epoch 31/60
100/100 [==============================] - ETA: 0s - loss: 0.0065 - accuracy: 0.9972
Epoch 00031: val_loss di

Epoch 56/60
100/100 [==============================] - ETA: 0s - loss: 0.0052 - accuracy: 0.9978
Epoch 00056: val_loss did not improve from 0.05926
100/100 [==============================] - 5s 52ms/step - loss: 0.0052 - accuracy: 0.9978 - val_loss: 0.1506 - val_accuracy: 0.9796 - lr: 1.0000e-04
Epoch 57/60
100/100 [==============================] - ETA: 0s - loss: 0.0052 - accuracy: 0.9978
Epoch 00057: val_loss did not improve from 0.05926
100/100 [==============================] - 5s 51ms/step - loss: 0.0052 - accuracy: 0.9978 - val_loss: 0.1513 - val_accuracy: 0.9773 - lr: 1.0000e-04
Epoch 58/60
100/100 [==============================] - ETA: 0s - loss: 0.0055 - accuracy: 0.9977
Epoch 00058: val_loss did not improve from 0.05926
100/100 [==============================] - 5s 51ms/step - loss: 0.0055 - accuracy: 0.9977 - val_loss: 0.1507 - val_accuracy: 0.9782 - lr: 1.0000e-04
Epoch 59/60
100/100 [==============================] - ETA: 0s - loss: 0.0053 - accuracy: 0.9977
Epoch 00059:

# Testing the Generated models

In [7]:
from tensorflow.keras.models import load_model
model = load_model(Params['model_loc'])

In [8]:
# test module is been uploaded from the data.py file
import sys
import os
sys.path.append(os.getcwd()+'/code')

from data import*

test_generator = testGenerator(Params['test_loc'],0,10,target_size=(256,256))
output = model.predict_generator(test_generator,10,verbose=1)
saveResult(Params['output_loc'],output)

Instructions for updating:
Please use Model.predict, which supports generators.
10/10 [==============================] - 0s 12ms/step


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0

In [9]:
os.getcwd()

'/home/erattakulangara/hpchome/DeepLearningAlgo/2022-Protocol-Adaptive_multiArtSegmentation'